## Getting started

thermodiff is not uploaded to PyPI. To install it you can do:

```shell
pip install git+https://github.com/SalvadorBrandolin/thermodiff
```

Obtaining derivatives of thermodynamic expressions is an important but not
trivial task. Derivatives of Residual Helmholtz free energy and Excess Gibbs
free energy models are used to evaluate of the thermodynamic properties of
mixtures, such as enthalpy, entropy, and pressure.

Moreover, sometimes it's required to obtain derivatives of expressions with 
thermodynamic-involved magnitudes/variables for the development of algorithms,
for example, for phase envelope calculations.

## Tutorial: Let's get the derivatives of the UNIQUAC model

The UNIQUAC (**uni**versal **qua**si**c**hemical) Excess Gibbs free energy 
model is given by:

$$ 
\frac{G^E}{RT} = \sum_k n_k \ln\frac{\phi_k}{x_k}
+ \frac{z}{2}\sum_k q_k n_k \ln\frac{\theta_k}{\phi_k}
- \sum_k q_k n_k \ln\left(\sum_l \theta_l \tau_{lk} \right)
$$

With:

$$x_k = \frac{n_k}{\sum_l n_l}$$

$$ \phi_k = \frac{r_k n_k}{\sum_l r_l n_l} $$

$$ \theta_k = \frac{q_k n_k}{\sum_l q_l n_l} $$

$$ \tau_{lk} = \exp \left[\frac{-\Delta U_{lk}}{R T} \right] $$

$$
\frac{-\Delta U_{lk}}{R T} = a_{lk}+\frac{b_{lk}}{T}+c_{lk}\ln T + d_{lk}T +
e_{lk}{T^2}
$$

A simplification could be made to avoid division by zero, replacing $\phi_k$
and $x_k$ on the Gibbs free energy. The result is:

$$ 
\frac{G^E}{RT} = \sum_k n_k \ln \left(\frac{n_T r_k}{\sum_l r_l n_l} \right)
+ \frac{z}{2}\sum_k q_k n_k \ln \left(\frac{q_k \sum_l r_l n_l}{r_k \sum_l q_l
n_l} \right) - \sum_k q_k n_k \ln\left(\sum_l \theta_l \tau_{lk} \right) 
$$

Or directly how we will use it in this tutorial:

$$ 
G^E = RT\left(\sum_k n_k \ln \left(\frac{n_T r_k}{\sum_l r_l n_l} \right)
+ \frac{z}{2}\sum_k q_k n_k \ln \left(\frac{q_k \sum_l r_l n_l}{r_k \sum_l q_l
n_l} \right) - \sum_k q_k n_k \ln\left(\sum_l \theta_l \tau_{lk} \right)\right)
$$

First thing to note is that in the expression above, all the subscripts are $k$
or $l$. This is for a personal preference. I like to differentiate the
expression respect to $n_i$ and $n_j$. If we built our expression to
differentiate using $i$ and $j$ subscripts, we wouldn't be able to
differentiate correctly.

:::{important}
If I was not clear. **NEVER** use $i$ and $j$ subscripts in the thermodynamic expressions you want to differentiate. Use $k$, $l$, $m$, etc.
:::


`thermodiff` already provide instantiated SymPy variables. Please use them
since internally when the $\frac{\partial}{\partial T}$ is performed, it will
use those instances.



In [1]:
from thermodiff import n, P, V, T  # mole number vector, pressure, volume, temperature
from thermodiff import k, l, m     # indexes
from thermodiff import R           # universal gas constant

import thermodiff as td

import sympy as sp  # we still could need sympy

### UNIQUAC Temperature function

When using `thermodiff`, you should imagine that later you will have to implement
those equations by yourself. So, differentiate as you code, by functions.
Moreover, a very recommended practice is to differentiate each term individually.
This avoids that SymPy heavily simplifies the expression to more non-readable
forms.

Following that advice, let's start by the easiest but instructive part,
$\tau_{lk}$. This force us to define more SymPy parameters: $a_{lk}$, $b_{lk}$,
$c_{lk}$, $d_{lk}$ and $e_{lk}$

In [2]:
# UNIQUAC interaction parameters
a_lk = sp.Symbol("a_{lk}")
b_lk = sp.Symbol("b_{lk}")
c_lk = sp.Symbol("c_{lk}")
d_lk = sp.Symbol("d_{lk}")
e_lk = sp.Symbol("e_{lk}")

# The SymPy expression
tau_lk_expr = sp.exp(a_lk + b_lk/T + c_lk*sp.ln(T) + d_lk*T + e_lk*T**2)

tau_lk_expr

exp(T**2*e_{lk} + T*d_{lk} + a_{lk} + c_{lk}*log(T) + b_{lk}/T)

Now we use the `thermodiff` class: `DiffPlz`

In [3]:
tau_lk_diffs = td.DiffPlz(
    expression=tau_lk_expr,
    internal_functions=[],
    indexes=[k, l, m],
    name=r"\tau_{lk}"
)

The arguments of the `DiffPlz` class are:

- `expression`: The SymPy expression to differentiate.
- `internal_functions`: A list of SymPy functions that are used internally in
  the expression. This is needed for cleaning the expression derivatives'
  LaTeX. We will see this later. By default, is an empty list.
- `indexes`: A list of SymPy variables that are used as indexes in the 
expression. By default, this list is [`k`, `l`, `m`]. It doesn't matter if you
put more variables than used in the expression, only you must not put less.
Don't forget that you can't use `i` and `j` as indexes in the expression.
- `name`: The name of the expression. This is used for the LaTeX representation
  of the expression. In this case, we will use $\tau_{lk}$


Inside the attributes of the `DiffPlz` instance we have the derivatives respect
to $T$:

In [4]:
tau_lk_diffs.dt

(2*T*e_{lk} + d_{lk} + c_{lk}/T - b_{lk}/T**2)*exp(T**2*e_{lk} + T*d_{lk} + a_{lk} + c_{lk}*log(T) + b_{lk}/T)

In [5]:
tau_lk_diffs.dt2

(2*e_{lk} - c_{lk}/T**2 + 2*b_{lk}/T**3)*exp(T**2*e_{lk} + T*d_{lk} + a_{lk} + c_{lk}*log(T) + b_{lk}/T) + (2*T*e_{lk} + d_{lk} + c_{lk}/T - b_{lk}/T**2)**2*exp(T**2*e_{lk} + T*d_{lk} + a_{lk} + c_{lk}*log(T) + b_{lk}/T)

Those are SymPy expressions, meaning that you can manipulate them as you wish
with all the power of SymPy. You can use `sympy.simplify`, `sympy.expand`, etc.

You will quickly notice that those temperature derivatives are horrible. They
are derivatives of an exponential function, so they have the form of:

$$
\frac{d e^{u}}{d T} = \frac{d u}{d T} e^u
$$

We can call the `clean_plz` method so the `DiffPlz` instance will try to
simplify those expressions. This will not always work, but in this case it 
does.

In [6]:
tau_lk_diffs.clean_plz()

The attributes are now replaced by "cleaned" expressions. Now we check the
derivatives of the $\tau_{lk}$ function again:

In [7]:
tau_lk_diffs.dt

(2*T*e_{lk} + d_{lk} + c_{lk}/T - b_{lk}/T**2)*\tau_{lk}(T)

In [8]:
tau_lk_diffs.dt2

(2*e_{lk} - c_{lk}/T**2 + 2*b_{lk}/T**3)*\tau_{lk}(T) + (2*T*e_{lk} + d_{lk} + c_{lk}/T - b_{lk}/T**2)**2*\tau_{lk}(T)

The last method we have you see is the `latex_readable_plz` method. This method
returns a dictionary with the LaTeX representation of the attributes. It will
clean all the arguments of the internal functions and will beautify the
derivatives. The idea is to use this dictionary to build the LaTeX in a
.tex/.md file and follow it to implement the equations in code.

In [9]:
ldict = tau_lk_diffs.latex_readable_plz()

ldict

{'dT': '\\left(2 T e_{lk} + d_{lk} + \\frac{c_{lk}}{T} - \\frac{b_{lk}}{T^{2}}\\right) \\tau_{lk}{\\left(T \\right)}',
 'dV': '0',
 'dP': '0',
 'dn_i': '0',
 'dT2': '\\left(2 e_{lk} - \\frac{c_{lk}}{T^{2}} + \\frac{2 b_{lk}}{T^{3}}\\right) \\tau_{lk}{\\left(T \\right)} + \\left(2 T e_{lk} + d_{lk} + \\frac{c_{lk}}{T} - \\frac{b_{lk}}{T^{2}}\\right)^{2} \\tau_{lk}{\\left(T \\right)}',
 'dV2': '0',
 'dP2': '0',
 'dn2': '0',
 'dTn': '0',
 'dVn': '0',
 'dPn': '0',
 'dTV': '0',
 'dTP': '0',
 'dVP': '0'}

Of course all derivatives are zero except the temperature derivatives. You
can print and copy those expressions to your LaTeX file. Here we are going to
represent them with IPython

In [11]:
from IPython.display import display, Math

display(Math(ldict["dT"]))
display(Math(ldict["dT2"]))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In this case is the same, but we will appreciate the difference with the next
examples.

### UNIQUAC $\theta_k$

The UNIQUAC $\theta_k$ function is very similar to the $\tau_{lk}$ function,
but still has some instructive differences. First, we need that the parameter
`q` to be index base. Second, we will see the function `sum_components`. This
function only provide a shortcut to make summation along the components
of the mixture.

 Let's do it:

In [15]:
q = sp.IndexedBase("q", shape=n.shape)

tetha_k = n[k] * q[k] / td.sum_components(n[l] * q[l], l)

tetha_k

n[k]*q[k]/Sum(n[l]*q[l], (l, 1, N_c))

As you can see, there is an internal variable `Nc` (number of components) that 
is used as the upper limit of the summation. You can import that variable if 
you want:

In [16]:
from thermodiff import nc

Now let's start differentiating:

In [17]:
tetha_k_diff = td.DiffPlz(
    expression=tetha_k,
    name=r"\theta_{k}"
)

In [18]:
tetha_k_diff.dni

Piecewise((-n[i]*q[i]**2/Sum(n[l]*q[l], (l, 1, N_c))**2 + q[i]/Sum(n[l]*q[l], (l, 1, N_c)), Eq(i, k)), (-n[k]*q[i]*q[k]/Sum(n[l]*q[l], (l, 1, N_c))**2, True))

In [19]:
tetha_k_diff.dnidnj

Piecewise((2*n[i]*q[i]**3/Sum(n[l]*q[l], (l, 1, N_c))**3 - 2*q[i]**2/Sum(n[l]*q[l], (l, 1, N_c))**2, Eq(i, j) & Eq(i, k)), (2*n[i]*q[i]**2*q[j]/Sum(n[l]*q[l], (l, 1, N_c))**3 - q[i]*q[j]/Sum(n[l]*q[l], (l, 1, N_c))**2, Eq(i, k)), (2*n[j]*q[i]*q[j]**2/Sum(n[l]*q[l], (l, 1, N_c))**3 - q[i]*q[j]/Sum(n[l]*q[l], (l, 1, N_c))**2, Eq(j, k)), (2*n[k]*q[i]*q[j]*q[k]/Sum(n[l]*q[l], (l, 1, N_c))**3, True))

As you can see, the compositional derivatives are piece wise expressions. This 
is obviously because the derivative depends on the subscripts. The first
derivative is a matrix and the second derivative is a tensor.

You could apply the `clean_plz` and `latex_readable_plz` methods to this
instance as previously shown.

### UNIQUAC $G^E$

As told before, its recommended to differentiate each term by sepparate. This
avoids that SymPy heavily simplifies the expression to more non-readable
forms. I will not do it here.

This is the first example of a expression that we need internal functions.
Specifically we need de $\tau_{lk}(T)$ and $\theta_l(n)$ functions.

First, the parameters:

In [25]:
r = sp.IndexedBase("r", shape=n.shape)
q = sp.IndexedBase("q", shape=n.shape)
z = sp.symbols("z")

Now the functions. First we will go with the $\theta_l(n)$. This is the most
complicated thing to understand, SymPy doesn't directly support indexed 
functions, so we have to use a custom `thermodiff` class.

In [27]:
theta_l = td.idx_function(r"\theta")(n[l])

theta_l

\theta(n[l])

Here we defined a function of `n[l]`. Since $\theta$ is function of $n$ we
symbolize the subindex of $\theta$ by the argument of the function. Don't use
the "l" in the function name because it will not be updated when the expression
is differentiated. In the final LaTeX, the subindex will be replaced from the
argument of the function to the subindex of the function.

In [28]:
tau_lk = td.idx_function(r"\tau")(l, k, T)

tau_lk

\tau(l, k, T)

Check what we have done with the $\tau_{lk}$ function. $\tau$ is only function
of temperature, so we don't use $n[l]$ or $n[k]$ in the arguments, we use
directly the indexes, so they can be updated when the expression is
differentiated.

In [31]:
Ge = R * T * (
    td.sum_components(n[k] * sp.ln(r[k] * td.sum_components(n[l], l) / td.sum_components(r[l] * n[l], l)), k)
    + z / 2 * (td.sum_components(q[k] * n[k] * sp.ln(q[k] * td.sum_components(r[l] * n[l], l) / (r[k] * td.sum_components(q[l] * n[l], l))), k))
    - td.sum_components(q[k] * n[k] * sp.ln(td.sum_components(theta_l * tau_lk, l)), k)
)

Ge

R*T*(z*Sum(log(q[k]*Sum(n[l]*r[l], (l, 1, N_c))/(r[k]*Sum(n[l]*q[l], (l, 1, N_c))))*n[k]*q[k], (k, 1, N_c))/2 + Sum(log(r[k]*Sum(n[l], (l, 1, N_c))/Sum(n[l]*r[l], (l, 1, N_c)))*n[k], (k, 1, N_c)) - Sum(log(Sum(\tau(l, k, T)*\theta(n[l]), (l, 1, N_c)))*n[k]*q[k], (k, 1, N_c)))

Nothing to help you here, you have to be brave and write the expression by
yourself.

Let's differentiate the expression:

In [32]:
ge_diff = td.DiffPlz(
    expression=Ge,
    internal_functions=[theta_l, tau_lk],
    indexes=[k, l],
    name=r"G^E"
)

Check that here we used the `internal_functions` argument to pass the
`theta_l` and `tau_lk` functions we used in the expression.

Let's check the derivatives: